## Trabalho Final de Previsão de Ativos Financeiros
### Análise Preditiva Avançada
### MBA Executivo em Business Analytics e Big Data - Campinas
### FGV


# Integrantes do Grupo: 
##### Adriana Marques Guidi, matrícula: A58318039
##### Érico Daniel Witzel dos Reis, matrícula: A58239101
##### Mário Elias Marinho Vieira, matrícula: A57860064
##### Rodrigo Kenji Makita Nakamura, matrícula: A57891680




In [1]:
####################################
### Importação de pacotes PYTHON ###
####################################
# geral
import pandas as pd
import numpy as np
from ta.utils import dropna
from ta.volatility import BollingerBands
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# rede neural
from tensorflow import keras
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import SGD
%matplotlib inline

#########################################
### Importação da base do mini-índice ###
#########################################
data = pd.read_csv('data.csv', sep='\t')

##########################################################################
### Definindo funções básicas - acuracia e separação de treino e teste ###
##########################################################################
# função acuracia
def acuracia(y_real, y_pred):
    return sum(y_real == y_pred) / len(y_pred)

# função para Separação de treino e teste
def split_treino_test(data):
    train = data[data['datetime'] < '2020-07-01']
    teste = data[data['datetime'] >= '2020-07-01']
    return train, teste

####################################
### Tratamentos básicos de datas ###
####################################
data['datetime'] = data['DATE'] + ' ' + data['TIME']
data['datetime'] = pd.to_datetime(data['datetime'])
# Ordena base de dados por datetime
data.sort_values(by = 'datetime', inplace = True)
# Mantém apenas as variáveis datetime, tickvol e close 
data = data[['datetime', 'TICKVOL', 'CLOSE']]

########################################
### Criando variável resposta target ###
########################################
data.rename(columns = {'CLOSE':'valor'}, inplace = True)
data.loc[:,'target_num'] = data['valor'].shift(-1)
data.loc[:,'target_diff'] = data['target_num']  - data['valor']
# cria variável target pela diferença entre a leitura do close atual 
# com a próxima leitura (leitura de 1 minuto posterior)
data.loc[:,'target'] = data['target_diff'].apply(lambda x: 1 if x>=10 else 0)
data = data.drop(columns = ['target_num', 'target_diff'])

########################################
### remoção da primeira linha do dia ###
########################################
data['primeira_do_dia'] = data['datetime'].diff() > pd.Timedelta(10, unit = 'hour')
data = data.loc[data['primeira_do_dia'] == False]
data = data[['datetime', 'valor', 'TICKVOL','target']].reset_index(drop = True)

###################################
### Criação de modelos baseline ###
###################################

#Criação de um modelo baseline utilizando 'naive', para servir como comparação para os modelos que serão criados. 
# Criação de variável target com lag
_, teste = split_treino_test(data)

teste_naive = teste.copy()

teste_naive['target_l1'] = teste_naive['target'].shift(1)
teste_naive=teste_naive.dropna()

# A predição é simplesmente o time step anterior
performance_baseline = acuracia(teste_naive['target'], teste_naive['target_l1'])
print("A acurácia do modelo Naive é " + str(round(100*performance_baseline,1)) + "%")

# Outra predição Naive seria assumir que todos os resultados são 0:
qtdade_zeros_base = 1-teste_naive['target'].mean()
print("A acurácia assumindo que todos são 0 é " + str(round(100*qtdade_zeros_base,1)) + "%")

A acurácia do modelo Naive é 52.5%
A acurácia assumindo que todos são 0 é 57.7%


In [2]:
##################################################################
### Criação de variáveis baseadas nos fechamentos e no tickvol ###
##################################################################

#########################################
### criando uma cópia da base inicial ###
#########################################
data_1 = data.copy()

###################################
### variáveis baseadas em close ###
###################################
d = pd.Series(data["valor"])

# variáveis de médias móveis 20 e 10 minutos
data_1['c_ma20'] = d.rolling(20).mean()
data_1['c_ma10'] = d.rolling(10).mean()

# comparando média móvel de 20 minutos com média móvel de 10 minutos
data_1['c_ma10/ma20'] = data_1['c_ma10']/data_1['c_ma20']
data_1['c_ma20_maior_ma10'] = np.where((data_1.c_ma20 > data_1.c_ma10),1,0)

# variáveis com máximo e mínimo de 20 e 10 minutos
data_1['c_max20'] = d.rolling(20).max()
data_1['c_max10'] = d.rolling(10).max()
data_1['c_min20'] = d.rolling(20).min()
data_1['c_min10'] = d.rolling(10).min()
data_1['c_max10/max20'] = data_1['c_max10']/data_1['c_max20']
data_1['c_max10/min20'] = data_1['c_max10']/data_1['c_min20']
data_1['c_min10/max20'] = data_1['c_min10']/data_1['c_max20']
data_1['c_min10/min20'] = data_1['c_min10']/data_1['c_min20']
data_1['c_min10/max10'] = data_1['c_min10']/data_1['c_max10']
data_1['c_min20/max20'] = data_1['c_min20']/data_1['c_max20']

# comparações entre mínimo, máximo e médias móveis
data_1['c_ma10/max20'] = data_1['c_ma10']/data_1['c_max20']
data_1['c_ma10/ma20'] = data_1['c_ma10']/data_1['c_ma20']
data_1['c_ma10/min20'] = data_1['c_ma10']/data_1['c_min20']
data_1['c_ma10/min10'] = data_1['c_ma10']/data_1['c_min10']
data_1['c_ma10/max10'] = data_1['c_ma10']/data_1['c_max10']
data_1['c_ma20/max20'] = data_1['c_ma20']/data_1['c_max20']
data_1['c_ma20/min20'] = data_1['c_ma20']/data_1['c_min20']
data_1['c_ma20/min10'] = data_1['c_ma20']/data_1['c_min10']
data_1['c_ma20/max10'] = data_1['c_ma20']/data_1['c_max10']
data_1['c_ma10_maior_max20'] = np.where((data_1.c_ma10 > data_1.c_max20),1,0)
data_1['c_ma20_menor_min10'] = np.where((data_1.c_ma20 < data_1.c_min10),1,0)
data_1['c_ma10_menor_min20'] = np.where((data_1.c_ma10 < data_1.c_min20),1,0)

# comparações entre ponto atual e máximos e mínimos de 10 e 20 minutos
data_1['c_valor_menor_min20'] = np.where((data_1.valor < data_1.c_min20),1,0)
data_1['c_valor_menor_min10'] = np.where((data_1.valor < data_1.c_min10),1,0)
data_1['c_valor_maior_max20'] = np.where((data_1.valor > data_1.c_max20),1,0)
data_1['c_valor_maior_max10'] = np.where((data_1.valor > data_1.c_max10),1,0)

# criação das bollinger bands considerando n = 20 minutos e 2 desvios padrão
indicator_bb = BollingerBands(close=data_1["valor"], n=20, ndev=2)
data_1['c_bb_bbp'] = indicator_bb.bollinger_pband()
data_1['c_bb_upper'] = indicator_bb.bollinger_hband_indicator()
data_1['c_bb_lower'] = indicator_bb.bollinger_lband_indicator()

# criação do fechamento lagado de até 3 minutos
data_1['c_valor_l1'] = data_1['valor'].shift(1)
data_1['c_valor_l2'] = data_1['valor'].shift(2)
data_1['c_valor_l3'] = data_1['valor'].shift(3)
# comparação entre o valor de fechamento atual e os fechamentos lagados
data_1['c_diff_l1'] = data_1['valor']  - data_1['c_valor_l1']
data_1['c_diff_l2'] = data_1['valor']  - data_1['c_valor_l2']
data_1['c_diff_l3'] = data_1['valor']  - data_1['c_valor_l3']
# comparação entre o valor de fechamento lagado de 1 minuto e fechamento lagado de 20 minutos
data_1['c_lag1/lag20'] = data_1['valor'].shift(1)/data_1['valor'].shift(20)
data_1['c_lag1/lag10'] = data_1['valor'].shift(1)/data_1['valor'].shift(10)
data_1['c_coef_ang20'] = (data_1['c_valor_l1'] - data_1['valor'].shift(20))/20
data_1['c_coef_ang10'] = (data_1['c_valor_l1'] - data_1['valor'].shift(10))/10

# quantidade de vezes que o índice fechou positivo ou negativo em relação ao minuto anterior 
# nos últimos 20 minutos e 10 minutos
x = 0
y = 0
for i in range(1,19):
    x = np.where((data_1['valor'].shift(i) > data_1['valor'].shift(i+1)),1,0) + x
    y = np.where((data_1['valor'].shift(i) < data_1['valor'].shift(i+1)),1,0) + y
    
a = 0
b = 0
for i in range(1,9):
    a = np.where((data_1['valor'].shift(i) > data_1['valor'].shift(i+1)),1,0) + a
    b = np.where((data_1['valor'].shift(i) < data_1['valor'].shift(i+1)),1,0) + b
    
data_1['c_qtde_vezes_positivo20'] = x
data_1['c_qtde_vezes_negativo20'] = y
data_1['c_qtde_vezes_positivo10'] = a
data_1['c_qtde_vezes_negativo10'] = b

# variáveis lagadas vs mínimos, média móvel 
data_1['c_lag1/min20'] = data_1['valor'].shift(1)/data_1['c_min20']
data_1['c_lag2/min20'] = data_1['valor'].shift(2)/data_1['c_min20']
data_1['c_lag3/min20'] = data_1['valor'].shift(3)/data_1['c_min20']
data_1['c_lag1/ma20'] = data_1['valor'].shift(1)/data_1['c_ma20']

# variável média vs máximo
data_1['c_ma3/max3'] = d.rolling(3).mean()/d.rolling(3).max()

# normalizando
media = data_1['c_ma20'].mean()
data_1['c_ma20'] = data_1['c_ma20']/media
media = data_1['c_ma10'].mean()
data_1['c_ma10'] = data_1['c_ma10']/media

media = data_1['c_max20'].mean()
data_1['c_max20'] = data_1['c_max20']/media
media = data_1['c_max10'].mean()
data_1['c_max10'] = data_1['c_max10']/media
media = data_1['c_min20'].mean()
data_1['c_min20'] = data_1['c_min20']/media
media = data_1['c_min10'].mean()
data_1['c_min10'] = data_1['c_min10']/media

media = data_1['c_bb_upper'].mean()
data_1['c_bb_upper'] = data_1['c_bb_upper']/media
media = data_1['c_bb_lower'].mean()
data_1['c_bb_lower'] = data_1['c_bb_lower']/media

media = data_1['c_valor_l1'].mean()
data_1['c_valor_l1'] = data_1['c_valor_l1']/media
media = data_1['c_valor_l2'].mean()
data_1['c_valor_l2'] = data_1['c_valor_l2']/media
media = data_1['c_valor_l3'].mean()
data_1['c_valor_l3'] = data_1['c_valor_l3']/media

media = data_1['c_diff_l1'].mean()
data_1['c_diff_l1'] = data_1['c_diff_l1']/media
media = data_1['c_diff_l2'].mean()
data_1['c_diff_l2'] = data_1['c_diff_l2']/media
media = data_1['c_diff_l3'].mean()
data_1['c_diff_l3'] = data_1['c_diff_l3']/media

media = data_1['c_coef_ang20'].mean()
data_1['c_coef_ang20'] = data_1['c_coef_ang20']/media
media = data_1['c_coef_ang10'].mean()
data_1['c_coef_ang10'] = data_1['c_coef_ang10']/media

media = data_1['c_qtde_vezes_positivo20'].mean()
data_1['c_qtde_vezes_positivo20'] = data_1['c_qtde_vezes_positivo20']/media
media = data_1['c_qtde_vezes_negativo20'].mean()
data_1['c_qtde_vezes_negativo20'] = data_1['c_qtde_vezes_negativo20']/media
media = data_1['c_qtde_vezes_positivo10'].mean()
data_1['c_qtde_vezes_positivo10'] = data_1['c_qtde_vezes_positivo10']/media
media = data_1['c_qtde_vezes_negativo10'].mean()
data_1['c_qtde_vezes_negativo10'] = data_1['c_qtde_vezes_negativo10']/media

media = data_1['valor'].mean()
data_1['valor'] = data_1['valor']/media

#####################################
### variáveis baseadas em tickvol ###
#####################################
a = pd.Series(data["TICKVOL"])

# variáveis de médias móveis 20 e 10 minutos
data_1['t_ma20'] = a.rolling(20).mean()
data_1['t_ma10'] = a.rolling(10).mean()
# comparando média móvel de 20 minutos com média móvel de 10 minutos
data_1['t_ma10/ma20'] = data_1['t_ma10']/data_1['t_ma20']
data_1['t_ma20_maior_ma10'] = np.where((data_1.t_ma20 > data_1.t_ma10),1,0)

# variáveis com máximo e mínimo de 20 e 10 minutos
data_1['t_max20'] = a.rolling(20).max()
data_1['t_max10'] = a.rolling(10).max()
data_1['t_min20'] = a.rolling(20).min()
data_1['t_min10'] = a.rolling(10).min()
data_1['t_max10/max20'] = data_1['t_max10']/data_1['t_max20']
data_1['t_max10/min20'] = data_1['t_max10']/data_1['t_min20']
data_1['t_min10/max20'] = data_1['t_min10']/data_1['t_max20']
data_1['t_min10/min20'] = data_1['t_min10']/data_1['t_min20']
data_1['t_min10/max10'] = data_1['t_min10']/data_1['t_max10']
data_1['t_min20/max20'] = data_1['t_min20']/data_1['t_max20']

# comparações entre mínimo, máximo e médias móveis
data_1['t_ma10/max20'] = data_1['t_ma10']/data_1['t_max20']
data_1['t_ma10/ma20'] = data_1['t_ma10']/data_1['t_ma20']
data_1['t_ma10/min20'] = data_1['t_ma10']/data_1['t_min20']
data_1['t_ma10/min10'] = data_1['t_ma10']/data_1['t_min10']
data_1['t_ma10/max10'] = data_1['t_ma10']/data_1['t_max10']
data_1['t_ma20/max20'] = data_1['t_ma20']/data_1['t_max20']
data_1['t_ma20/min20'] = data_1['t_ma20']/data_1['t_min20']
data_1['t_ma20/min10'] = data_1['t_ma20']/data_1['t_min10']
data_1['t_ma20/max10'] = data_1['t_ma20']/data_1['t_max10']
data_1['t_ma10_maior_max20'] = np.where((data_1.t_ma10 > data_1.t_max20),1,0)
data_1['t_ma20_menor_min10'] = np.where((data_1.t_ma20 < data_1.t_min10),1,0)
data_1['t_ma10_menor_min20'] = np.where((data_1.t_ma10 < data_1.t_min20),1,0)

# comparações entre ponto atual e máximos e mínimos de 10 e 20 minutos
data_1['t_valor_menor_min20'] = np.where((data_1.TICKVOL < data_1.t_min20),1,0)
data_1['t_valor_menor_min10'] = np.where((data_1.TICKVOL < data_1.t_min10),1,0)
data_1['t_valor_maior_max20'] = np.where((data_1.TICKVOL > data_1.t_max20),1,0)
data_1['t_valor_maior_max10'] = np.where((data_1.TICKVOL > data_1.t_max10),1,0)

# criação das bollinger bands considerando n = 20 minutos e 2 desvios padrão
indicator_aa = BollingerBands(close=data_1["TICKVOL"], n=20, ndev=2)
data_1['t_bb_bbp'] = indicator_aa.bollinger_pband()
data_1['t_bb_upper'] = indicator_aa.bollinger_hband_indicator()
data_1['t_bb_lower'] = indicator_aa.bollinger_lband_indicator()

# criação do tickvol lagado de até 3 minutos
data_1['t_valor_l1'] = data_1['TICKVOL'].shift(1)
data_1['t_valor_l2'] = data_1['TICKVOL'].shift(2)
data_1['t_valor_l3'] = data_1['TICKVOL'].shift(3)
# comparação entre o tickvol atual e o tickvol lagado
data_1['t_diff_l1'] = data_1['TICKVOL']  - data_1['t_valor_l1']
data_1['t_diff_l2'] = data_1['TICKVOL']  - data_1['t_valor_l2']
data_1['t_diff_l3'] = data_1['TICKVOL']  - data_1['t_valor_l3']
# comparação entre o tickvol lagado de 1 minuto e tickvol lagado de 20 minutos
data_1['t_lag1/lag20'] = data_1['TICKVOL'].shift(1)/data_1['TICKVOL'].shift(20)
data_1['t_lag1/lag10'] = data_1['TICKVOL'].shift(1)/data_1['TICKVOL'].shift(10)
data_1['t_coef_ang20'] = (data_1['t_valor_l1'] - data_1['TICKVOL'].shift(20))/20
data_1['t_coef_ang10'] = (data_1['t_valor_l1'] - data_1['TICKVOL'].shift(10))/10

# quantidade de vezes que o tickvol fechou positivo ou negativo em relação ao minuto anterior 
# nos últimos 20 minutos e 10 minutos
c = 0
d = 0
for i in range(1,19):
    c = np.where((data_1['TICKVOL'].shift(i) > data_1['TICKVOL'].shift(i+1)),1,0) + c
    d = np.where((data_1['TICKVOL'].shift(i) < data_1['TICKVOL'].shift(i+1)),1,0) + d
    
e = 0
f = 0
for i in range(1,9):
    e = np.where((data_1['TICKVOL'].shift(i) > data_1['TICKVOL'].shift(i+1)),1,0) + e
    f = np.where((data_1['TICKVOL'].shift(i) < data_1['TICKVOL'].shift(i+1)),1,0) + f
    
data_1['t_qtde_vezes_positivo20'] = c
data_1['t_qtde_vezes_negativo20'] = d
data_1['t_qtde_vezes_positivo10'] = e
data_1['t_qtde_vezes_negativo10'] = f

# variáveis lagadas vs mínimos, média móvel 
data_1['t_lag1/min20'] = data_1['TICKVOL'].shift(1)/data_1['t_min20']
data_1['t_lag2/min20'] = data_1['TICKVOL'].shift(2)/data_1['t_min20']
data_1['t_lag3/min20'] = data_1['TICKVOL'].shift(3)/data_1['t_min20']
data_1['t_lag1/ma20'] = data_1['TICKVOL'].shift(1)/data_1['t_ma20']

# variável média vs máximo
data_1['t_ma3/max3'] = a.rolling(3).mean()/a.rolling(3).max()

# normalizando
media = data_1['t_ma20'].mean()
data_1['t_ma20'] = data_1['t_ma20']/media
media = data_1['t_ma10'].mean()
data_1['t_ma10'] = data_1['t_ma10']/media

media = data_1['t_max20'].mean()
data_1['t_max20'] = data_1['t_max20']/media
media = data_1['t_max10'].mean()
data_1['t_max10'] = data_1['t_max10']/media
media = data_1['t_min20'].mean()
data_1['t_min20'] = data_1['t_min20']/media
media = data_1['t_min10'].mean()
data_1['t_min10'] = data_1['t_min10']/media

media = data_1['t_bb_upper'].mean()
data_1['t_bb_upper'] = data_1['t_bb_upper']/media
media = data_1['t_bb_lower'].mean()
data_1['t_bb_lower'] = data_1['t_bb_lower']/media

media = data_1['t_valor_l1'].mean()
data_1['t_valor_l1'] = data_1['t_valor_l1']/media
media = data_1['t_valor_l2'].mean()
data_1['t_valor_l2'] = data_1['t_valor_l2']/media
media = data_1['t_valor_l3'].mean()
data_1['t_valor_l3'] = data_1['t_valor_l3']/media

media = data_1['t_diff_l1'].mean()
data_1['t_diff_l1'] = data_1['t_diff_l1']/media
media = data_1['t_diff_l2'].mean()
data_1['t_diff_l2'] = data_1['t_diff_l2']/media
media = data_1['t_diff_l3'].mean()
data_1['t_diff_l3'] = data_1['t_diff_l3']/media

media = data_1['t_coef_ang20'].mean()
data_1['t_coef_ang20'] = data_1['t_coef_ang20']/media
media = data_1['t_coef_ang10'].mean()
data_1['t_coef_ang10'] = data_1['t_coef_ang10']/media

media = data_1['t_qtde_vezes_positivo20'].mean()
data_1['t_qtde_vezes_positivo20'] = data_1['t_qtde_vezes_positivo20']/media
media = data_1['t_qtde_vezes_negativo20'].mean()
data_1['t_qtde_vezes_negativo20'] = data_1['t_qtde_vezes_negativo20']/media
media = data_1['t_qtde_vezes_positivo10'].mean()
data_1['t_qtde_vezes_positivo10'] = data_1['t_qtde_vezes_positivo10']/media
media = data_1['t_qtde_vezes_negativo10'].mean()
data_1['t_qtde_vezes_negativo10'] = data_1['t_qtde_vezes_negativo10']/media

media = data_1['TICKVOL'].mean()
data_1['TICKVOL'] = data_1['TICKVOL']/media

#####################
### target lagado ###
#####################
data_1['valor_target_l1'] = data_1['target'].shift(1)
data_1['valor_target_l2'] = data_1['target'].shift(2)
data_1['valor_target_l3'] = data_1['target'].shift(3)
data_1['valor_target_l4'] = data_1['target'].shift(4)
data_1['valor_target_l5'] = data_1['target'].shift(5)
data_1['valor_target_l6'] = data_1['target'].shift(6)
data_1['valor_target_l7'] = data_1['target'].shift(7)
data_1['valor_target_l8'] = data_1['target'].shift(8)
data_1['valor_target_l9'] = data_1['target'].shift(9)
data_1['valor_target_l10'] = data_1['target'].shift(10)
data_1['valor_target_l11'] = data_1['target'].shift(11)
data_1['valor_target_l12'] = data_1['target'].shift(12)
data_1['valor_target_l13'] = data_1['target'].shift(13)
data_1['valor_target_l14'] = data_1['target'].shift(14)
data_1['valor_target_l15'] = data_1['target'].shift(15)
data_1['valor_target_l16'] = data_1['target'].shift(16)
data_1['valor_target_l17'] = data_1['target'].shift(17)
data_1['valor_target_l18'] = data_1['target'].shift(18)
data_1['valor_target_l19'] = data_1['target'].shift(19)
data_1['valor_target_l20'] = data_1['target'].shift(20)

###################################
### variáveis baseadas em datas ###
###################################
# dia da semana
data_1['dia_semana'] = pd.DatetimeIndex(data_1['datetime']).weekday # 0 é segunda #
# hora do dia
data_1['hora_do_dia'] = pd.DatetimeIndex(data_1['datetime']).hour
# período do mês, inicial, meio ou final
data_1['dia_do_mes'] = np.where((pd.DatetimeIndex(data_1['datetime']).day < 10),1,
                               np.where((pd.DatetimeIndex(data_1['datetime']).day < 20),2,3))
# período do dia
data_1['periodo_dia'] = np.where((pd.DatetimeIndex(data_1['datetime']).hour < 12),1,
                               np.where((pd.DatetimeIndex(data_1['datetime']).hour < 15),2,3))

###############
### ajustes ###
###############
# retira observações vazias
data_1 = data_1.dropna()
data_1.reset_index(drop = True, inplace = True)

###########################################################
### geração de variável aleatória para Cross validation ###
###########################################################
rng = np.random.default_rng()
data_1['cv'] = pd.DataFrame(rng.integers(1, 6, size=(96204, 1)))

# separação entre treino e teste, variáveis explicativas e target
treino_1, teste_1 = split_treino_test(data_1)
X_treino_1 = treino_1.copy().drop(columns = ['target', 'datetime'])
X_teste_1 = teste_1.copy().drop(columns = ['target', 'datetime','cv'])

y_treino_1 = treino_1[['target','cv']]
y_teste_1 = teste_1[['target']]

data_1

,datetime,valor,TICKVOL,target,c_ma20,c_ma10,c_ma10/ma20,c_ma20_maior_ma10,c_max20,c_max10,...,valor_target_l16,valor_target_l17,valor_target_l18,valor_target_l19,valor_target_l20,dia_semana,hora_do_dia,dia_do_mes,periodo_dia,cv
0,2019-12-09 09:20:00,1.137678,0.333476,0,1.137225,1.137527,1.000274,0,1.135332,1.136222,...,0.0,1.0,0.0,1.0,1.0,0,9,1,1,3
1,2019-12-09 09:21:00,1.137371,0.342275,0,1.137302,1.137517,1.000198,0,1.135332,1.136222,...,1.0,0.0,1.0,0.0,1.0,0,9,1,1,5
2,2019-12-09 09:22:00,1.137269,0.339812,1,1.137319,1.137492,1.000160,0,1.135332,1.136222,...,0.0,1.0,0.0,1.0,0.0,0,9,1,1,3
3,2019-12-09 09:23:00,1.137422,0.271708,0,1.137348,1.137487,1.000130,0,1.135332,1.136222,...,1.0,0.0,1.0,0.0,1.0,0,9,1,1,5
4,2019-12-09 09:24:00,1.137371,0.165242,0,1.137358,1.137466,1.000103,0,1.135332,1.136222,...,1.0,1.0,0.0,1.0,0.0,0,9,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96199,2020-08-31 17:50:00,1.019978,0.893435,0,1.020441,1.020558,1.000123,0,1.019355,1.020154,...,1.0,0.0,0.0,0.0,1.0,0,17,3,3,3
96200,2020-08-31 17:51:00,1.019671,0.372895,1,1.020398,1.020558,1.000165,0,1.019355,1.020154,...,1.0,1.0,0.0,0.0,0.0,0,17,3,3,1
96201,2020-08-31 17:52:00,1.020029,0.199382,0,1.020372,1.020543,1.000175,0,1.019355,1.020154,...,0.0,1.0,1.0,0.0,0.0,0,17,3,3,3
96202,2020-08-31 17:53:00,1.019467,0.216804,1,1.020318,1.020481,1.000168,0,1.019355,1.020154,...,0.0,0.0,1.0,1.0,0.0,0,17,3,3,1


In [3]:
####################################################################################################
### Treinamento e validação através de k-folds para definição dos hiperparâmetros da rede neural ###
####################################################################################################

# Criação dos folds - explicativas
X_treino_cv1 = X_treino_1[X_treino_1['cv'] != 1]
X_treino_cv1 = X_treino_cv1.copy().drop(columns = ['cv'])
X_teste_cv1 = X_treino_1[X_treino_1['cv'] == 1]
X_teste_cv1 = X_teste_cv1.copy().drop(columns = ['cv'])

X_treino_cv2 = X_treino_1[X_treino_1['cv'] != 2]
X_treino_cv2 = X_treino_cv2.copy().drop(columns = ['cv'])
X_teste_cv2 = X_treino_1[X_treino_1['cv'] == 2]
X_teste_cv2 = X_teste_cv2.copy().drop(columns = ['cv'])

X_treino_cv3 = X_treino_1[X_treino_1['cv'] != 3]
X_treino_cv3 = X_treino_cv3.copy().drop(columns = ['cv'])
X_teste_cv3 = X_treino_1[X_treino_1['cv'] == 3]
X_teste_cv3 = X_teste_cv3.copy().drop(columns = ['cv'])

X_treino_cv4 = X_treino_1[X_treino_1['cv'] != 4]
X_treino_cv4 = X_treino_cv4.copy().drop(columns = ['cv'])
X_teste_cv4 = X_treino_1[X_treino_1['cv'] == 4]
X_teste_cv4 = X_teste_cv4.copy().drop(columns = ['cv'])

X_treino_cv5 = X_treino_1[X_treino_1['cv'] != 5]
X_treino_cv5 = X_treino_cv5.copy().drop(columns = ['cv'])
X_teste_cv5 = X_treino_1[X_treino_1['cv'] == 5]
X_teste_cv5 = X_teste_cv5.copy().drop(columns = ['cv'])

# Criação dos folds - target
y_treino_cv1 = y_treino_1[y_treino_1['cv'] != 1]
y_treino_cv1 = y_treino_cv1.copy().drop(columns = ['cv'])
y_teste_cv1 = y_treino_1[y_treino_1['cv'] == 1]
y_teste_cv1 = y_teste_cv1.copy().drop(columns = ['cv'])

y_treino_cv2 = y_treino_1[y_treino_1['cv'] != 2]
y_treino_cv2 = y_treino_cv2.copy().drop(columns = ['cv'])
y_teste_cv2 = y_treino_1[y_treino_1['cv'] == 2]
y_teste_cv2 = y_teste_cv2.copy().drop(columns = ['cv'])

y_treino_cv3 = y_treino_1[y_treino_1['cv'] != 3]
y_treino_cv3 = y_treino_cv3.copy().drop(columns = ['cv'])
y_teste_cv3 = y_treino_1[y_treino_1['cv'] == 3]
y_teste_cv3 = y_teste_cv3.copy().drop(columns = ['cv'])

y_treino_cv4 = y_treino_1[y_treino_1['cv'] != 4]
y_treino_cv4 = y_treino_cv4.copy().drop(columns = ['cv'])
y_teste_cv4 = y_treino_1[y_treino_1['cv'] == 4]
y_teste_cv4 = y_teste_cv4.copy().drop(columns = ['cv'])

y_treino_cv5 = y_treino_1[y_treino_1['cv'] != 5]
y_treino_cv5 = y_treino_cv5.copy().drop(columns = ['cv'])
y_teste_cv5 = y_treino_1[y_treino_1['cv'] == 5]
y_teste_cv5 = y_teste_cv5.copy().drop(columns = ['cv'])

###########################################
### Criação da estrutura da Rede Neural ###
###########################################
modelo = keras.Sequential()
# criação da camada de entrada, ativação relu
modelo.add(keras.layers.Dense(128, activation='relu', input_shape=(128,)))
# criação da camada escondida com 3 neurônios, ativação relu
modelo.add(keras.layers.Dense(3, activation='relu'))
# criação da camada de saída, ativação sigmóide para saída em probabilidade
modelo.add(keras.layers.Dense(1, activation='sigmoid'))

# alteração do tamanho do learning rate do otimizador SGD
sgd = SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
modelo.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

########################################
### treino sem cv1, avaliação em cv1 ###
########################################
historico1 = modelo.fit(x=X_treino_cv1, y=y_treino_cv1,
                       epochs=10,
                       batch_size=64, 
                       verbose = 0,
                       validation_data=(X_teste_cv1, y_teste_cv1)
                      )
print('treino k-fold1')
score_n1 = modelo.evaluate(X_treino_cv1, y_treino_cv1, verbose=0)
print(score_n1)
print('teste validação k-fold1')
score_n1_t = modelo.evaluate(X_teste_cv1, y_teste_cv1, verbose=0)
print(score_n1_t)

########################################
### treino sem cv2, avaliação em cv2 ###
########################################
historico2 = modelo.fit(x=X_treino_cv2, y=y_treino_cv2,
                       epochs=10,
                       batch_size=64, 
                       verbose = 0,
                       validation_data=(X_teste_cv2, y_teste_cv2)
                      )
print('treino k-fold2')
score_n2 = modelo.evaluate(X_treino_cv2, y_treino_cv2, verbose=0)
print(score_n2)
print('teste validação k-fold2')
score_n2_t = modelo.evaluate(X_teste_cv2, y_teste_cv2, verbose=0)
print(score_n2_t)

########################################
### treino sem cv3, avaliação em cv3 ###
########################################
historico3 = modelo.fit(x=X_treino_cv3, y=y_treino_cv3,
                       epochs=10,
                       batch_size=64, 
                       verbose = 0,
                       validation_data=(X_teste_cv3, y_teste_cv3)
                      )
print('treino k-fold3')
score_n3 = modelo.evaluate(X_treino_cv3, y_treino_cv3, verbose=0)
print(score_n3)
print('teste validação k-fold3')
score_n3_t = modelo.evaluate(X_teste_cv3, y_teste_cv3, verbose=0)
print(score_n3_t)

########################################
### treino sem cv4, avaliação em cv4 ###
########################################
historico4 = modelo.fit(x=X_treino_cv4, y=y_treino_cv4,
                       epochs=10,
                       batch_size=64, 
                       verbose = 0,
                       validation_data=(X_teste_cv4, y_teste_cv4)
                      )
print('treino k-fold4')
score_n4 = modelo.evaluate(X_treino_cv4, y_treino_cv4, verbose=0)
print(score_n4)
print('teste validação k-fold4')
score_n4_t = modelo.evaluate(X_teste_cv4, y_teste_cv4, verbose=0)
print(score_n4_t)

########################################
### treino sem cv5, avaliação em cv5 ###
########################################
historico5 = modelo.fit(x=X_treino_cv5, y=y_treino_cv5,
                       epochs=10,
                       batch_size=64, 
                       verbose = 0,
                       validation_data=(X_teste_cv5, y_teste_cv5)
                      )
print('treino k-fold5')
score_n5 = modelo.evaluate(X_treino_cv5, y_treino_cv5, verbose=0)
print(score_n5)
print('teste validação k-fold5')
score_n5_t = modelo.evaluate(X_teste_cv5, y_teste_cv5, verbose=0)
print(score_n5_t)

# avaliação da acurácia nos k-folds
print('Acurácia - Teste Validação')
(score_n1_t[1]+score_n2_t[1]+score_n3_t[1]+score_n4_t[1]+score_n5_t[1])/5

treino k-fold1
[0.6889737844467163, 0.5756904482841492]
teste validação k-fold1
[0.6891381740570068, 0.573017418384552]
treino k-fold2
[0.686564564704895, 0.5741598606109619]
teste validação k-fold2
[0.6860184073448181, 0.5791801810264587]
treino k-fold3
[0.6847860217094421, 0.575209379196167]
teste validação k-fold3
[0.6848282217979431, 0.574955403804779]
treino k-fold4
[0.6834331154823303, 0.5766840577125549]
teste validação k-fold4
[0.6847869157791138, 0.5691068172454834]
treino k-fold5
[0.6832878589630127, 0.5740514993667603]
teste validação k-fold5
[0.6821707487106323, 0.579579770565033]
Acurácia - Teste Validação


0.5751679182052613

In [15]:
###############################
### Criação do modelo final ###
###############################

# remoção da variável auxiliar cv
X_treino_final = X_treino_1.copy().drop(columns = ['cv'])
y_treino_final = y_treino_1.copy().drop(columns = ['cv'])

# Criação da estrutura da Rede Neural
modelo = keras.Sequential()
# criação da camada de entrada, ativação relu
modelo.add(keras.layers.Dense(128, activation='relu', input_shape=(128,))) 
# criação da camada escondida com 3 neurônios, ativação relu
modelo.add(keras.layers.Dense(3, activation='relu'))
# criação da camada de saída, ativação sigmóide para saída em probabilidade
modelo.add(keras.layers.Dense(1, activation='sigmoid'))

# alteração do tamanho do learning rate do otimizador SGD
sgd = SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
modelo.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

# treino final
print('treino final')
historico1 = modelo.fit(x=X_treino_final, y=y_treino_final,
                       epochs=10,
                       batch_size=64
                      )

treino final
Epoch 1/20
1137/1137 [==============================] - 3s 3ms/step - loss: 56.1856 - accuracy: 0.5739
Epoch 2/20
1137/1137 [==============================] - 3s 2ms/step - loss: 0.6922 - accuracy: 0.5752
Epoch 3/20
1137/1137 [==============================] - 3s 2ms/step - loss: 0.6917 - accuracy: 0.5752
Epoch 4/20
1137/1137 [==============================] - 3s 2ms/step - loss: 0.6911 - accuracy: 0.5752
Epoch 5/20
1137/1137 [==============================] - 3s 2ms/step - loss: 0.6906 - accuracy: 0.5752
Epoch 6/20
1137/1137 [==============================] - 3s 2ms/step - loss: 0.6901 - accuracy: 0.5752
Epoch 7/20
1137/1137 [==============================] - 3s 2ms/step - loss: 0.6897 - accuracy: 0.5752
Epoch 8/20
1137/1137 [==============================] - 3s 2ms/step - loss: 0.6892 - accuracy: 0.5752
Epoch 9/20
1137/1137 [==============================] - 3s 2ms/step - loss: 0.6888 - accuracy: 0.5752
Epoch 10/20
1137/1137 [==============================] - 3s 3ms/step

In [16]:
##################################
### Avaliação na amostra teste ###
##################################

# avaliação na amostra teste
score_n = modelo.evaluate(X_teste_1, y_teste_1, verbose=0)
print('resultado na amostra de teste')
score_n

resultado na amostra de teste


[0.6852574348449707, 0.5771803259849548]